In [1]:
import segmentation_models_pytorch as smp
model = smp.UnetPlusPlus(
    encoder_name="efficientnet-b0",
    in_channels=3,
    encoder_depth=3, 
    decoder_channels=(64, 32, 16),
    classes=1,
    )

/home/ljb/miniconda3/envs/sde/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from torch import nn
import torch
import math
import segmentation_models_pytorch as smp


class timeUnetPlusPlus(smp.UnetPlusPlus):
    def __init__(self, model_channels, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # self.timeEmb
        self.model_channels = model_channels
        time_embed_dim = model_channels * 4
        self.time_embed = nn.Sequential(
            nn.Linear(model_channels, time_embed_dim),
            nn.SiLU(),
            nn.Linear(time_embed_dim, time_embed_dim),
        )
        self.lns = nn.ModuleList()
        self.lns.append(nn.Linear(time_embed_dim, 2))
        self.lns.append(nn.Linear(time_embed_dim, 32))
        self.lns.append(nn.Linear(time_embed_dim, 24))
        self.lns.append(nn.Linear(time_embed_dim, 40))
        
        self.name = 'timeUnetPlusPlus'
        self.initialize()

    def timestep_embedding(timesteps, dim, max_period=10000):
        """
        Create sinusoidal timestep embeddings.
        :param timesteps: a 1-D Tensor of N indices, one per batch element.
                        These may be fractional.
        :param dim: the dimension of the output.
        :param max_period: controls the minimum frequency of the embeddings.
        :return: an [N x dim] Tensor of positional embeddings.
        """
        half = dim // 2
        freqs = torch.exp(
            -math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half
        ).to(device=timesteps.device)
        args = timesteps[:, None].float() * freqs[None]
        embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
        if dim % 2:
            embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
        return embedding


    def forward(self, x, timesteps):        
        emb = self.time_embed(self.timestep_embedding(timesteps, self.model_channels))
        print(emb.shape)
        self.check_input_shape(x)
        features = self.encoder(x)
        for index, f in enumerate(features):
            temp_emb = self.lns[index](emb)
            while len(temp_emb.shape) < len(f.shape):
                temp_emb = temp_emb[..., None]
            print(f.shape, temp_emb.shape)
            f = f + temp_emb
        decoder_output = self.decoder(*features)

        masks = self.segmentation_head(decoder_output)

        if self.classification_head is not None:
            labels = self.classification_head(features[-1])
            return masks, labels

        return masks
    
    
model = timeUnetPlusPlus(
    model_channels=64,
    encoder_name="efficientnet-b0",
    in_channels=2,
    encoder_depth=3, 
    decoder_channels=(64, 32, 16),
    classes=1,
    ).to('cuda')

In [13]:
x = torch.rand(100, 2, 32, 32).to('cuda')
t = torch.rand(100).to('cuda')
model(x, t).shape

TypeError: arange() received an invalid combination of arguments - got (dtype=torch.dtype, end=Tensor, start=int, ), but expected one of:
 * (Number end, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (Number start, Number end, *, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (Number start, Number end, Number step, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
